# Jupyter Notebook to clean install all the contracts


### Necessary packages 

In [75]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Set Up Environment

In [76]:
network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom
contracts_file_custom = 'yeouido contracts_20210219095435.pkl'
#contracts_20201109120425

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.237.205.52:9000/",       "nid": 3},
"local":   {"iconservice": "http://localhost:9000/",           "nid": 3}
}

env = connections[network]

In [77]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from datetime import datetime
from repeater import retry
from shutil import make_archive
import requests
import json
import pickle as pkl
import csv
import os
from pprint import pprint
from getpass import getpass
from lendingPoolCore.Math import *

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"


with open(contracts_file_custom, 'rb') as f:
    contracts = pkl.load(f)

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result




In [78]:
pprint(contracts)

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33',
 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286',
 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e',
 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c',
 'oICX': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
 'oIUSDC': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'oUSDb': 'cx66788ee4dca134edae9ea9b8cc98c4e8073e644b',
 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'rewards': 'cx3e9b

In [39]:
contracts['usdb']="cxaa068556df80f9917ef146e889f0b2c4b13ab634"
contracts['oUSDb']="cx1aef9c66e508cfc32282e27e645ace726e831bd3"

In [29]:

deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to("cx9a5a9c116379ecb9e4aadb423955fc9351771aa6")\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('delegation'))\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0x7bc16e1148e6bc7342ab1c575e17892172d4f3e66a2882f215eb7ea7de5ad5d5', 'blockHeight': 12217995, 'blockHash': '0x6003bcd25699c83e92590d79d682871ff7a25dc88ad3830803450d3cbd1e8f1b', 'txIndex': 1, 'to': 'cx9a5a9c116379ecb9e4aadb423955fc9351771aa6', 'scoreAddress': 'cx9a5a9c116379ecb9e4aadb423955fc9351771aa6', 'stepUsed': 1675653600, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1675653600, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

### Required service

In [79]:
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

# Importing wallet

### Importing from private key

In [82]:

private="a691ef7d5601f9b5be4f9b9d80215159ea6ff0b88003e3d34e078d40e778b39a"
deployer_wallet = KeyWallet.load(bytes.fromhex(private))
deployer_wallet.get_address()


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

In [83]:
icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce")

19139991313150298909795

### Importing wallet from keystore file

In [8]:
# Set to the address of the keystore file and give the password when prompted
password = getpass()
deployer_wallet = KeyWallet.load("./keystores/keystore_test1.json",password)
deployer_wallet.get_address()
icon_service.get_balance(deployer_wallet.get_address())

708876575999000000000000000

In [84]:
private2="b3d4e18d733f05dc21f326ba4fe150514874327c738c4a19ddd357d0e8e2efed"
deployer_wallet2 = KeyWallet.load(bytes.fromhex(private2))
print(deployer_wallet2.get_address())

private3="c3d5e9afe753a07c983bb14e34f73907e40a997793ce2f7d2895cddec27f81c6"
deployer_wallet3 = KeyWallet.load(bytes.fromhex(private3))
print(deployer_wallet3.get_address())

private4="2ffb9ef51826d555189d4804161058036cf87ae8290f1a308a79d045f6cdd428"
deployer_wallet4 = KeyWallet.load(bytes.fromhex(private4))
print(deployer_wallet4.get_address())
wallets=[deployer_wallet2,deployer_wallet3,deployer_wallet4]

hxd29037f1eb8c3f0e8630d797edc7410507c0089a
hx66ee5acace86d9ce23127599b7c9daae70c9d7c1
hx65902dd4139d04765534a4539319be874ad7aa0b


# Deploying all contracts


In [9]:
#contracts=AutoVivification()

deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx', 'liquidationManager']
# for directory in {"score"}:
directory=os.getcwd()
for item in deploy:
    params = {}
    if item == "sample_token":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "sicx":
        params = {'_initialSupply':500000000, '_decimals': 18}
    elif item == "oToken":
        params = {"_name":"BridgeUSDInterestToken","_symbol":"oUSDb"}
    deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(item))\
        .params(params)\
        .build()

    step_limit = icon_service.estimate_step(deploy_transaction) + 100000
    
    signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    contracts[item] = res.get('scoreAddress', '')

#deploy oICX 
params = {"_name":"ICXinterestToken","_symbol":"oICX"}
deploy_transaction = DeployTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(GOVERNANCE_ADDRESS)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content('oToken'))\
        .params(params)\
        .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(res)
contracts['oICX'] = res.get('scoreAddress', '')

now = datetime.utcnow()

filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename=network+'_'+filename
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

contracts

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending tra

{'addressProvider': 'cxc595228d12cc267c748026a6629af41060654dab',
 'feeProvider': 'cxb76322c7df895a14e47bc14b4aa21566b3edc069',
 'lendingPool': 'cxf44d5c379d7b458fe4bba1ecb0f8fd4a54e9d65c',
 'lendingPoolCore': 'cx515613f46e2b333f6bdfa5d7fd1bf93fa3043e14',
 'lendingPoolDataProvider': 'cxfdff702158ae5bfce3f25a4c040b850d96e25af8',
 'oICX': 'cxf872138faf25dfc5a1216a7a85cad24b8172d988',
 'oToken': 'cx9732efc2d75905dfe4d8bc2d055c2cf226391d00',
 'priceOracle': 'cx1fbce991b255545bcd31634ab6beda5605201f46',
 'sample_token': 'cxc88281539407ee6d766e58d302f27c50d517783c',
 'sicx': 'cx400639d22b6bce4def6e7d7be2bc36e7a39192ee',
 'liquidationManager': 'cx847bb94d386b34af9ba86dfdcace651dbd5af822'}

In [12]:
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'delegation': 'cx5ecc21ade5e4a90586bde6451da3b0d733dd11d6',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'governance': 'cx9e0ecfd9cc9a0c1cee8fc4a4701e970554835f54',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'oUSDb': 'cx1aef9c66e508cfc32282e27e645ace726e831bd3',
 'omm_token': 'cxff036606b11066a142262431b089e8d0af145bef',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'rewards': 'cxcccee742d4c1b7a3a84a20a41691d302e25ab3a0',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'snapshot': 

In [18]:
params = {"_name":"USDCInterestToken","_symbol":"oIUSDC","_decimals":6}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(GOVERNANCE_ADDRESS)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .params(params)\
    .content(gen_deploy_data_content('oToken'))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

result = get_tx_result(tx_hash)
print(result)
contracts['oIUSDC']= result.get('scoreAddress','')



Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xc9a123a741d0dda7565aaf94d430b3fe6799c2c7e4766ba63aedc7ff96d4ec13', 'blockHeight': 13316260, 'blockHash': '0x2f80ee431fc2a4d786190c4aa7c7c08e800b784fa7a6990dc5b2204dc3705e22', 'txIndex': 1, 'to': 'cx0000000000000000000000000000000000000000', 'scoreAddress': 'cx147b13e34e1041852ffbd0e4d03523c64ad9b5a7', 'stepUsed': 1146144000, 'stepPrice': 10000000000, 'cumulativeStepUsed': 1146144000, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [19]:
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'delegation': 'cx5ecc21ade5e4a90586bde6451da3b0d733dd11d6',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'governance': 'cx9e0ecfd9cc9a0c1cee8fc4a4701e970554835f54',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oIUSDC': 'cx147b13e34e1041852ffbd0e4d03523c64ad9b5a7',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'oUSDC': 'cxcb083f613855dc19533f85b35a6605d7a9539cda',
 'oUSDb': 'cx1aef9c66e508cfc32282e27e645ace726e831bd3',
 'omm_token': 'cxff036606b11066a142262431b089e8d0af145bef',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'rewards': 'cxcccee

In [20]:
now = datetime.utcnow()

filename = f'contracts_{now.year}{str(now.month).zfill(2)}{str(now.day).zfill(2)}{str(now.hour).zfill(2)}{str(now.minute).zfill(2)}{str(now.second).zfill(2)}.pkl'
filename="shubh_yeouido_contracts_20210112084238.pkl"
print(f'saving contracts list as {filename}')
with open(filename, 'wb') as f:
    pkl.dump(contracts, f)

saving contracts list as shubh_yeouido_contracts_20210112084238.pkl


## Save updated contract

# ICX transfer

In [17]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae")\
    .nid(3) \
    .nonce(100) \
    .value(1000000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)


# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0x3c546494f559d4ea9d34534f13387c74aa80658c67ac2630ce1adaf514e1bc86',
 'blockHeight': 1425621,
 'blockHash': '0xc6a2b22340f3c2756027f1926f731d1eaa78ed6c65653e92e85f58e7ee2d2b66',
 'txIndex': 1,
 'to': 'hx49c8a4c31e84b9abe7f8c8395eb7133b2151aeae',
 'stepUsed': 100000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 100000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [26]:
params = {"_owner":"hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['iusdc'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("balanceOf")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x2816bd52f88cf443e5d0133ae314b39e6901e51dc0295a0642066b7c02c2bc41', 'blockHeight': 12723774, 'blockHash': '0xc49da4fbe608653ada7ece5674446f8cd72f3ebb76235fa7e03d5272e4a7d52b', 'txIndex': 1, 'to': 'cx65f639254090820361da483df233f6d0e69af9b7', 'stepUsed': 142400, 'stepPrice': 10000000000, 'cumulativeStepUsed': 142400, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Updating a contract

In [211]:
#deploy = ['addressProvider', 'feeProvider', 'lendingPool', 'lendingPoolCore', 'lendingPoolDataProvider','oToken','priceOracle','sample_token', 'sicx', 'oICX', 'liquidationManager']
#for contract_ in deploy:


contract_ = 'priceOracle'
if contract_ == 'oICX':
    update = contracts['oICX']
    contract_ = 'oToken' 
elif contract_ == 'oUSDb':
    update = contracts['oUSDb']
    contract_ = 'oToken' 
elif contract_ == 'oIUSDC':
    update = contracts['oIUSDC']
    contract_ = 'oToken' 
else:
    update = contracts[contract_]
#update = contracts[contract_]
print('update',contract_, update)

deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(update)\
    .nid(NID)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content(contract_))\
    .build()

step_limit = icon_service.estimate_step(deploy_transaction) + 100000

signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)


update priceOracle cxe76fd817fae3b485ae6e4fea2cf88d050d379b63
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x0a3e58c4e97c4bb3036654c10945cd263a4ab4ab70d61e6de8be7341b25e4798',
 'blockHeight': 13408722,
 'blockHash': '0x98b94671320b27d272c0bd007d498e8f605903e7659006b012c0c836c6a02d88',
 'txIndex': 1,
 'to': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'scoreAddress': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'stepUsed': 1651607200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1651607200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Configuring SCOREs's parameters

In [15]:
settings = [{'contract': 'lendingPool', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPool', 'method': 'setDataProvider', 'params':{'_address': contracts['lendingPoolDataProvider']}},
            {'contract': 'lendingPool', 'method': 'setFeeProvider', 'params':{'_address': contracts['feeProvider']}},
            {'contract': 'lendingPool', 'method': 'setLiquidationManagerAddress','params': {'_address':contracts['liquidationManager']}},
            {'contract': 'lendingPool', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'lendingPool', 'method': 'setOICXAddress', 'params':{'_address': contracts['oICX']}},
            {'contract': 'feeProvider', 'method': 'setLoanOriginationFeePercentage', 'params':{'_percentage': 1*10**15}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sample_token'],'_sym':"USDb"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['sicx'],'_sym':"ICX"}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolCoreAddress', 'params':{'_address': contracts['lendingPoolCore']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setOracleAddress', 'params':{'_address': contracts['priceOracle']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLendingPoolAddress', 'params':{'_address': contracts['lendingPool']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setLiquidationAddress', 'params':{'_address': contracts['liquidationManager']}},
            {'contract': 'lendingPoolDataProvider', 'method': 'setStakingAddress', 'params':{'_address': contracts['staking']}},
            {'contract': 'oToken', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oToken', 'method': 'setReserveAddress', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'oToken', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oToken', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'ICX','_quote':'USD','_rate':5*10**17}},
            {'contract': 'addressProvider', 'method': 'setLendingPool', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'addressProvider', 'method': 'setLendingPoolDataProvider', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['sample_token']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oToken']}},
            {'contract': 'addressProvider', 'method': 'setsICX', 'params':{'_address':contracts['sicx']}},
            {'contract': 'addressProvider', 'method': 'setoICX', 'params':{'_address':contracts['oICX']}},
            {'contract': 'addressProvider', 'method': 'setStaking', 'params':{'_address':contracts['staking']}
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sample_token'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}},
            {'contract': 'lendingPool', 'method': 'setSICXAddress', 'params':{'_address': contracts['sicx']}},  
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['sicx'],"optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":f"0{'0'*17}","slopeRate1":f"8{'0'*16}","slopeRate2":f"2{'0'*18}"} ]}},
            {'contract': 'oICX', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oICX', 'method': 'setReserveAddress', 'params':{'_address':contracts['sicx']}},
            {'contract': 'oICX', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oICX', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'liquidationManager', 'method': 'setDataProviderAddress', 'params': {'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'liquidationManager', 'method': 'setCoreAddress', 'params': {'_address':contracts['lendingPoolCore']}},
            {'contract': 'liquidationManager', 'method': 'setOracleAddress','params': {'_address':contracts['priceOracle']}},
            {'contract': 'liquidationManager', 'method': 'setFeeProviderAddress','params': {'_address':contracts['feeProvider']}},]

In [18]:
contracts['iusdc']="cx65f639254090820361da483df233f6d0e69af9b7"
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oIUSDC': 'cx17f109dc5aed3e97ba9309c8a4508e24e594a11b',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'oUSDb': 'cx1aef9c66e508cfc32282e27e645ace726e831bd3',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c',
 'usdb': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'}


In [41]:
usdb_settings = [
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['usdb'],'_sym':"USDb"}},
            {'contract': 'oUSDb', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oUSDb', 'method': 'setReserveAddress', 'params':{'_address':contracts['usdb']}},
            {'contract': 'oUSDb', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oUSDb', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDb','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setUSDb', 'params':{'_address':contracts['usdb']}},
            {'contract': 'addressProvider', 'method': 'setoUSDb', 'params':{'_address':contracts['oUSDb']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['usdb'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}}]
for sett in iusdc_settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))


In [25]:
iusdc_settings = [
            {'contract': 'lendingPoolDataProvider', 'method': 'setSymbol', 'params':{'_reserveAddress': contracts['iusdc'],'_sym':"USDC"}},
            {'contract': 'oIUSDC', 'method': 'setCoreAddress', 'params':{'_address':contracts['lendingPoolCore']}},
            {'contract': 'oIUSDC', 'method': 'setReserveAddress', 'params':{'_address':contracts['iusdc']}},
            {'contract': 'oIUSDC', 'method': 'setDataProviderAddress', 'params':{'_address':contracts['lendingPoolDataProvider']}},
            {'contract': 'oIUSDC', 'method': 'setLendingPoolAddress', 'params':{'_address':contracts['lendingPool']}},
            {'contract': 'priceOracle', 'method': 'set_reference_data', 'params':{'_base':'USDC','_quote':'USD','_rate':1*10**18}},
            {'contract': 'addressProvider', 'method': 'setIUSDC', 'params':{'_address':contracts['iusdc']}},
            {'contract': 'addressProvider', 'method': 'setoIUSDC', 'params':{'_address':contracts['oIUSDC']}},
            {'contract': 'lendingPoolCore', 'method': 'setReserveConstants', 'params' :{"_constants": [{"reserve":contracts['iusdc'],"optimalUtilizationRate":f"8{'0'*17}","baseBorrowRate":f"2{'0'*16}","slopeRate1":f"4{'0'*16}","slopeRate2":f"1{'0'*18}"} ]}}]
for sett in iusdc_settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSymbol, with parameters {'_reserveAddress': 'cx65f639254090820361da483df233f6d0e69af9b7', '_sym': 'USDC'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
{'txHash': '0x3a49675ef5033d3b3a9de28554e076c9dcbf23afa6fa296fee77026c77b78d3e', 'blockHeight': 13316556, 'blockHash': '0x3e144fc73eefd39a2c7130a073230a1721280b6ff0eaa6b397c2452477129b49', 'txIndex': 1, 'to': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582', 'stepUsed': 147320, 'stepPrice': 10000000000, 'cumulativeStepUsed': 147320, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [42]:
for sett in settings:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash))



Calling setSymbol, with parameters {'_reserveAddress': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', '_sym': 'USDb'} on the lendingPoolDataProvider contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...
{'txHash': '0xd43a330d7b602ada103f04e9ead2f5a67245a406375ac62ccb3adff500ed92bc', 'blockHeight': 11798983, 'blockHash': '0xda80af67633b4675cfad2ed0b12e192cb3e760824354bbebacd3f91545b37885', 'txIndex': 1, 'to': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582', 'stepUsed': 148280, 'stepPrice': 10000000000, 'cumulativeStepUsed': 148280, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [44]:
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'oUSDb': 'cx1aef9c66e508cfc32282e27e645ace726e831bd3',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c',
 'usdb': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634'}


## Adding IUSDC reserve in the lendingPoolCore


In [22]:
params ={"_reserve": {"reserveAddress":contracts['iusdc'],"oTokenAddress":contracts['oIUSDC'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"500000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"6","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xd2009b33326913c9441d9e91aa342343b793c45845dbb7f6998f692dec7eaf64',
 'blockHeight': 13316406,
 'blockHash': '0x953a7ea51129e6927a88359824cd68642c83952e50d476e8263913c19391fd34',
 'txIndex': 1,
 'to': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'stepUsed': 247960,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 247960,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding USDb reserve in LendingPoolCore



In [43]:
params ={"_reserve": {"reserveAddress":contracts['usdb'],"oTokenAddress":contracts['oUSDb'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"330000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa35d028251ad66d1c2cb2b822a1391e31c633a39d26eab5bc392cf3f0653abbb',
 'blockHeight': 11799042,
 'blockHash': '0x87f6f3dda85ee80559e2b80faef715bb8136dd339e27c16a7a7231f687bbebbe',
 'txIndex': 1,
 'to': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'stepUsed': 276800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

## Adding ICX reserve

In [18]:
params ={"_reserve": {"reserveAddress":contracts['sicx'],"oTokenAddress":contracts['oICX'],"totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"330000000000000000","liquidationThreshold":"650000000000000000","liquidationBonus":"100000000000000000","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore']) \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
# print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0x62d984b4d8378d1e0cf0c2067ab7ed32da82f111b4cb2755586879824c6434ef',
 'blockHeight': 1870960,
 'blockHash': '0xa2b37b48ba8f5b076c482ec35d6f822e1c053f20121632ee1a4ce947b73f65a1',
 'txIndex': 1,
 'to': 'cx515613f46e2b333f6bdfa5d7fd1bf93fa3043e14',
 'stepUsed': 276800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 276800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

# Transfering USDb to a wallet

In [25]:
params = {"_to": "hxb6084c615cc424ef2fc5329c07e2e6d2fbc7f0f8", "_value": 10000 * 10 ** 18}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xc07b16c637e63952a12a3b781feb0850c60c1bbf9a749d471d9dc64d8d4725a9',
 'blockHeight': 11297686,
 'blockHash': '0xec3e2eb18730a98e4388d0efe781d08ba83163bc4a56c9fa83723f63a4d94685',
 'txIndex': 1,
 'to': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
 'stepUsed': 161460,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 161460,
 'eventLogs': [{'scoreAddress': 'cx19be1be7b4e7750863edf966faac3f42dec21025',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce',
    'hxb6084c615cc424ef2fc5329c07e2e6d2fbc7f0f8',
    '0x21e19e0c9bab2400000'],
   'data': ['0x4e6f6e65']}],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x02\x00\x00\x00\x00\

# Depositing USDb from a wallet

In [197]:
data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 1000000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sample_token'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xc05dae4ef871b2b015ca36b03fdebd06129f10ed6148db660d7fe2881b1dd330',
 'blockHeight': 13407667,
 'blockHash': '0xb517b45b1ac6f55ac274cc8dee924de712ad160e61cc88238bb02072bc298ea1',
 'txIndex': 1,
 'to': 'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75',
 'stepUsed': 797640,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 797640,
 'eventLogs': [{'scoreAddress': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75',
    '0x0',
    '0x470de4df820000'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x3635c9adc5dea00000']},
  {'scoreAddress': 'c

In [198]:
dataJson = {'method':'deposit','params':{'amount':10 * 10**6}}
data = json.dumps(dataJson).encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 10 * 10**6, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['iusdc'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x49a72f7efe90ca0beaef31739be7088da1fd6f7d93a9c2d9205f471888f50641',
 'blockHeight': 13407676,
 'blockHash': '0x593ef039633cb46e119205ad6d2c5f285e293bac829f2d666d5e5463bea51591',
 'txIndex': 1,
 'to': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'stepUsed': 781340,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 781340,
 'eventLogs': [{'scoreAddress': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx65f639254090820361da483df233f6d0e69af9b7',
    '0x0',
    '0x470de4df820000'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x989680']},
  {'scoreAddress': 'cx5bdb716d9d5

# Borrowing USDb

In [199]:
params ={"_reserve": contracts['sample_token'], "_amount":10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85', 'stepLimit': '0x1f81dc', 'timestamp': '0x5bc134ce7a36b', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75', '_amount': '0x8ac7230489e80000'}}, 'signature': '9RIWjHLkHJ2mWu0i0dnD0anL/jKDqfrvnCFoNMFEfJUa0qCVKaVyiVaq0oWy5DDFbjo1EiEb1zxv8IrBsnm38QE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xa230582347ec4048cf8e2a3629c6c58fd30cb6f45353e63ef123a14676c110e9',
 'blockHeight': 13407692,
 'blockHash': '0x69b4bc676549afb0cd10c325397beb2b17acdff6a0400289a79b64e417b17d47',
 'txIndex': 1,
 'to': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'stepUsed': 1964860,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1964860,
 'eventLogs': [{'scoreAddress': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
   'indexed': ['FundReceived(int,Address,int)',
    '0x0',
    'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75',
    '0x5bc134cfb5c65'],
   'data': []},
  {'scoreAddress': 'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
    'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
    '0x0'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
   'indexed': ['DaoFundTransfer(int,Address,Address)',
    '0x0',
    'cxc88e39248bf428d3ab0b9c9fbafef9ff423c

In [200]:
params ={"_reserve": contracts['iusdc'], "_amount":1*10**6}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85', 'stepLimit': '0x2021a0', 'timestamp': '0x5bc134d93ac3d', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'borrow', 'params': {'_reserve': 'cx65f639254090820361da483df233f6d0e69af9b7', '_amount': '0xf4240'}}, 'signature': 'pgAA0iGZ++pQMCEM0IiHJ70YCwPGmpyDr3LYMQJ30b9dkNCR3OQ+AQDfe8guwddtx0Z4ZyCb0bJlFkRgpfBYdwA='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x2cdbaa24b50b11837916c09993a706e16f6650fcaec18675601b11f7bbcccb58',
 'blockHeight': 13407698,
 'blockHash': '0x7453c0432a355ad310f2e825131517897b993d12832250d89d2c7e32be2b69e8',
 'txIndex': 1,
 'to': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'stepUsed': 2005760,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 2005760,
 'eventLogs': [{'scoreAddress': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
   'indexed': ['FundReceived(int,Address,int)',
    '0x0',
    'cx65f639254090820361da483df233f6d0e69af9b7',
    '0x5bc134db34541'],
   'data': []},
  {'scoreAddress': 'cx65f639254090820361da483df233f6d0e69af9b7',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
    'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
    '0x0'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
   'indexed': ['DaoFundTransfer(int,Address,Address)',
    '0x0',
    'cx65f639254090820361da483df233f6d0e69a

# Reedeming/Withdrawing USDb

In [201]:
params ={"_amount": 10*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058', 'stepLimit': '0x1ebbe4', 'timestamp': '0x5bc134e863935', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'redeem', 'params': {'_amount': '0x8ac7230489e80000'}}, 'signature': 'RQVDJE94BxczSXn1TF+skvwW1ZvrA9vT2st+Vkxw7rVNyeQTO841UBkA4ygSy3QJthed1wpCeXTzofB2xHpdVAE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x9e6525e9d55f2374571e6b22fc1a1beb01c2cee1758f963ac8824fab195e8861',
 'blockHeight': 13407706,
 'blockHash': '0xf97b64c0136d1694c01033915f20b25f18a7c711672d67194d449bb0a9eac788',
 'txIndex': 1,
 'to': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'stepUsed': 1914180,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1914180,
 'eventLogs': [{'scoreAddress': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x262400bb68']},
  {'scoreAddress': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x8ac7230489e80000']},
  {'scoreAddress': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75',
    '0xa989db0362d9',
    '0x48d93c1b747989'],
   'data': ['0xde0b6b3b12795b1', '0xde0b6b7e447a3c1']},
 

In [202]:
params ={"_amount": 1*10**6 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oIUSDC']) \
    .nid(NID) \
    .nonce(100) \
    .method("redeem")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c', 'stepLimit': '0x1e9d1c', 'timestamp': '0x5bc134f958f3f', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'redeem', 'params': {'_amount': '0xf4240'}}, 'signature': 'uEpQ0whgdYztMeb4MYVnI5muNWf+H6nfvytz3lBNo499jekz0d2DxpWgcXILJrTeT4Zv+VjShRTTQ6L4mRSmGQE='}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xb2a9a89d03392c94d5c6b7d807fcca5989c8165800e582c234481a15e2d70822',
 'blockHeight': 13407715,
 'blockHash': '0x0508b1a835e66d51a3345098c0865cbd991e6f494d7332451aefc44ca1e82fa2',
 'txIndex': 1,
 'to': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'stepUsed': 1906300,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1906300,
 'eventLogs': [{'scoreAddress': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
   'indexed': ['Burn(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0xf4240']},
  {'scoreAddress': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx65f639254090820361da483df233f6d0e69af9b7',
    '0x91443a517b8e3',
    '0x5acaa472ed38e4'],
   'data': ['0xde0b6b437fac1af', '0xde0b6b9edef1408']},
  {'scoreAddress': '

# Repaying USDb

In [44]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 1000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 1000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['usdb'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x0d86fe2be4a8f505e67972f6cc99569d23d4eb2d7b4815487d9e22933d995bd3',
 'blockHeight': 13261179,
 'blockHash': '0x5e8a3b3e9b431b8dac28c9866cfd90876289a905c5c216891f28d2ea0303f8be',
 'txIndex': 1,
 'to': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
 'stepUsed': 1134780,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1134780,
 'eventLogs': [{'scoreAddress': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
    'cxb216bd790f1ca2f77fd5d6110879c694380417f8',
    '0xb293cb393'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
   'indexed': ['DaoFundTransfer(int,Address,Address)',
    '0xb293cb393',
    'cxaa068556df80f9917ef146e889f0b2c4b13ab634',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': []},
  {'scoreAddress': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
   'indexed': ['PrintData(str,int,int,int)',
    'params ch

In [203]:
data = "{\"method\": \"repay\", \"params\": {\"amount\": 1000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 1000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['iusdc'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0x0a46495c8d08fb0cc96f5ee3c09656d9e2e611d308f40f0f7a7d271ec5db88c8',
 'blockHeight': 13407721,
 'blockHash': '0xbdbb898c6eb863bd984b78a4c499ac40ed1629d15232d0df7b1d36ff13ba603b',
 'txIndex': 1,
 'to': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'stepUsed': 1094600,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1094600,
 'eventLogs': [{'scoreAddress': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
   'indexed': ['FundReceived(int,Address,int)',
    '0x0',
    'cx65f639254090820361da483df233f6d0e69af9b7',
    '0x5bc135072f7bf'],
   'data': []},
  {'scoreAddress': 'cx65f639254090820361da483df233f6d0e69af9b7',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
    'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
    '0x0'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
   'indexed': ['DaoFundTransfer(int,Address,Address)',
    '0x0',
    'cx65f639254090820361da483df233f6d0e69a

In [23]:
print(icon_service.get_balance("hx91bf040426f226b3bfcd2f0b5967bbb0320525ce"))

9450000385630198414715


# Depositing ICX from Wallet

In [205]:
params = {"_amount": 50 * 10 ** 18}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address()) \
    .to(contracts['lendingPool']) \
    .nid(3) \
    .step_limit(1000000000) \
    .nonce(100) \
    .value(50 * 10 ** 18) \
    .method("deposit") \
    .params(params) \
    .build() 

signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x8e9b6fdd7bbab0f0871ff7240e8f81bdb8b1fe1b46c0f0c7b504bf6fabdc17b1',
 'blockHeight': 13408312,
 'blockHash': '0x3c9a8da3b4a7f9fe22a14f6ef5235d925d42c1e80949dbeaf518d2936898b53e',
 'txIndex': 1,
 'to': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'stepUsed': 1035000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1035000,
 'eventLogs': [{'scoreAddress': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
    'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c',
    '0x2b5e3af16b1880000'],
   'data': []},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
    'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
    '0x2aa149d1cfb07e5f7'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
   'indexed': ['Mint(Address,int,bytes)',
    '

In [89]:
pprint(contracts)

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33',
 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286',
 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e',
 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c',
 'oICX': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
 'oIUSDC': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'oUSDb': 'cx66788ee4dca134edae9ea9b8cc98c4e8073e644b',
 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'rewards': 'cx3e9b

# Withdrawing ICX 

1017313011655355613


In [212]:
settings_priceOracle =[{'contract':  'priceOracle', 'method': 'setBandOracle','params': {'_address':"cx61a36e5d10412e03c907a507d1e8c6c3856d9964"}},
                       {'contract':  'priceOracle', 'method': 'toggleOraclePriceBool','params':{}}]
for sett in settings_priceOracle:
    print(f'\n')
    print(f'Calling {sett["method"]}, with parameters {sett["params"]} on the {sett["contract"]} contract.')
    transaction = CallTransactionBuilder()\
        .from_(deployer_wallet.get_address())\
        .to(contracts[sett['contract']])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(sett['method'])\
        .params(sett['params'])\
        .build()
    signed_transaction = SignedTransaction(transaction, deployer_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    tx_hash
    print(get_tx_result(tx_hash)['status'])



Calling setBandOracle, with parameters {'_address': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


Calling toggleOraclePriceBool, with parameters {} on the priceOracle contract.
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
1


In [163]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['staking'])\
    .method("getTodayRate")\
    .build()
todayRate = int(icon_service.call(_call),0)
pprint(todayRate)
# ICX_amount = 10 * ICX
# sicx_amount =exaDiv(ICX_amount,todayRate)
# print(sicx_amount)
ICX_amount = 5 * ICX
sicx_amount =exaDiv(ICX_amount,todayRate)
params = {"_amount": sicx_amount,"_waitForUnstaking":0}
call_transaction = CallTransactionBuilder() \
    .from_(deployer_wallet.get_address())\
    .to(contracts['oICX'])\
    .nid(3) \
    .step_limit(10000000) \
    .nonce(100) \
    .method("redeem") \
    .params(params) \
    .build()
signed_transaction = SignedTransaction(call_transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)
        

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x4a4a1b266d36dffd717fda739d6672b34823fc560d3618ff652a7f9736f2c1ef',
 'blockHeight': 13406675,
 'blockHash': '0xd4ad38c2ce177438baba1342ceedb56d3e2a44c5afefca670ed917d8de6723a7',
 'txIndex': 1,
 'to': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
 'stepUsed': 1356820,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1356820,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [52]:
pprint(contracts)

{'addressProvider': 'cxdcbae76661c0edbf968d048c3bba3311bde57e37',
 'feeProvider': 'cx52555a5dda72b98a3ed692b58761f68af9486073',
 'lendingPool': 'cx348b53ff3bb5c6411f8c3e7082cde123bd86c67e',
 'lendingPoolCore': 'cxc15058ed8a262ff3b3a2400786848a3c32c274a5',
 'lendingPoolDataProvider': 'cx45bd68dd4f903160f4c7f2c43791fa741debb582',
 'liquidationManager': 'cx001794d12ea9170f4bb7a620e0a03a157f35a429',
 'oICX': 'cx7dcf7232084a085507bf2685f785589ebb0b52a1',
 'oToken': 'cx77d34d57f7ea300637058ef65babc52da91376a4',
 'priceOracle': 'cx0b57375df6a814ca3f589e91b15015f135fbd046',
 'sample_token': 'cx9da823297393ecb193030b983ce334f25af20aac',
 'sicx': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'}


# Borrowing sicx

In [186]:
#borrow sicx
params ={"_reserve": contracts['sicx'], "_amount":1*10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPool']) \
    .nid(NID) \
    .nonce(100) \
    .method("borrow")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32001, 'message': 'integer division or modulo by zero'}


JSONRPCException: {'code': -32001, 'message': 'integer division or modulo by zero'} (Json rpc error)

# Repay sicx

In [191]:

data = "{\"method\": \"repay\", \"params\": {\"amount\": 1000000000000000000}}".encode("utf-8")
params = {"_to": contracts['lendingPool'], "_value": 1000000000000000000, "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['sicx'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 2 seconds...


{'txHash': '0xb42b1a3f14280d5b3a147b544784a609c46c2971a051b467a808ee02435f5acd',
 'blockHeight': 13407553,
 'blockHash': '0xaa7eb0947e56ebc0c12406d96eca629d9669799f9f236868db102db68616a8ef',
 'txIndex': 1,
 'to': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
 'stepUsed': 1081440,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1081440,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

# Readonly methods to get data

### Getting all the addresses

In [86]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['addressProvider'])\
    .method("getAllAddresses")\
    .build()
response = icon_service.call(_call)
pprint(response)

{'collateral': {'IUSDC': 'cx65f639254090820361da483df233f6d0e69af9b7',
                'USDb': 'cxc88e39248bf428d3ab0b9c9fbafef9ff423c3a75',
                'sICX': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281'},
 'oTokens': {'oICX': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
             'oIUSDC': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
             'oUSDb': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058'},
 'systemContract': {'LendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
                    'LendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
                    'Staking': 'cxadd9557a07d4aab925dd04b7a6da4ea1d67bef9c'}}


### Getting specific reserve data

In [219]:
#this returns the reserve data of USDb
params = {'_reserve': contracts['iusdc']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if values=="0x1" or values =="0x0" or "Address" in key:
        continue
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0) 
    else:
        response[key]=int(values,0)

pprint(response)

{'availableLiquidity': 8999000000000000000,
 'availableLiquidityUSD': 8999000000000000000,
 'baseLTVasCollateral': 500000000000000000,
 'borrowCumulativeIndex': 1000000036677660037,
 'borrowRate': 20005561110493210,
 'borrowingEnabled': '0x1',
 'decimals': 6,
 'exchangePrice': 1000000000000000000,
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': 1614166023665599,
 'liquidationBonus': 100000000000000000,
 'liquidationThreshold': 650000000000000000,
 'liquidityCumulativeIndex': 1000000003398232710,
 'liquidityRate': 2002556444654,
 'oTokenAddress': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'reserveAddress': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'totalBorrows': 1001000000000000,
 'totalBorrowsUSD': 1001000000000000,
 'totalLiquidity': 9000001000000000000,
 'totalLiquidityUSD': 9000001000000000000,
 'usageAsCollateralEnabled': '0x1'}


# Reserve Account Data

In [54]:
#this returns the reserve data of USDb

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveAccountData")\
    .build()
response = icon_service.call(_call)
# print(response)
for key,values in response.items():
    response[key]=int(values,0)/10**18
        

pprint(response)

{'availableLiquidityBalanceUSD': 100.50524603864064,
 'totalBorrowsBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 100.50524603864064,
 'totalLiquidityBalanceUSD': 100.50524603864064}


### Getting data of all reserves

In [220]:
#this returns the reserve data of all the reserves
params = {'_reserve': contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if v=="0x1" or v =="0x0" or "Address" in k:
            continue
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'ICX': {'availableLiquidity': 93.3822546228794,
         'availableLiquidityUSD': 170.0861786002422,
         'baseLTVasCollateral': 0.5,
         'borrowCumulativeIndex': 1.0000000140892038,
         'borrowRate': 9.35473294785e-07,
         'borrowingEnabled': '0x1',
         'decimals': 1.8e-17,
         'exchangePrice': 1.7904,
         'isActive': '0x1',
         'isFreezed': '0x0',
         'lastUpdateTimestamp': 1614167209007701,
         'liquidationBonus': 0.1,
         'liquidationThreshold': 0.65,
         'liquidityCumulativeIndex': 1.0000000002839329,
         'liquidityRate': 5.906994e-12,
         'oTokenAddress': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
         'reserveAddress': 'cxb7eda227d9ed9c6fc2a74aa5dd7e9a3913f6c281',
         'sICXRate': 1.0173130116553557,
         'totalBorrows': 0.001000013954911065,
         'totalBorrowsUSD': 0.001821422633503913,
         'totalLiquidity': 93.38325463683431,
         'totalLiquidityUSD': 170.0880000228757,
         '

In [196]:
pprint(contracts)

{'addressProvider': 'cx797bc63e7b27758816910cb35f0fa1f546b57c33',
 'daoFund': 'cxb2e669fbb8b94f2a4525345f22e466933b8d2075',
 'delegation': 'cx24e32367425660a67c91d6e3c1077914df4e0286',
 'feeProvider': 'cxd55f27870c9b219272ea4fc31d5d7ec8dc78a97e',
 'governance': 'cx672132f49f0ef02969cc6b223613206e46ff34b1',
 'iusdc': 'cx65f639254090820361da483df233f6d0e69af9b7',
 'lendingPool': 'cx0be0870a036ef7a08997bd12e1cd0bfed2dd5d85',
 'lendingPoolCore': 'cx2d844c9613b5f864f5dace7fed82997c4c62ef01',
 'lendingPoolDataProvider': 'cx88ae036091243916d87cbc3b11f39e317dcd46ad',
 'liquidationManager': 'cx21b6ee09cd144b2a778bdcf69a46ef2cf898ff1c',
 'oICX': 'cx9dfe6300788178da4ac43e7eb7114380f5addc69',
 'oIUSDC': 'cx5bdb716d9d5531927f9cab4dfd26937a595ad52c',
 'oToken': 'cxf3e8b176a276a1d6bde232638656ed835f5cb058',
 'oUSDb': 'cx66788ee4dca134edae9ea9b8cc98c4e8073e644b',
 'ommToken': 'cxa96491850d5dd69efa5d64afa9138fd4a66cd348',
 'priceOracle': 'cxe76fd817fae3b485ae6e4fea2cf88d050d379b63',
 'rewards': 'cx3e9b

In [41]:
params = {'_reserve': contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolCore'])\
    .method("getReserveConstants")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key != "reserve":
        response[key]=int(values,0)/10**18

   
pprint(response)

{'baseBorrowRate': 0.02,
 'optimalUtilizationRate': 0.8,
 'reserve': 'cx94afc91e5b1c749afa20ec63c7a91669296417ee',
 'slopeRate1': 0.04,
 'slopeRate2': 1.0}


### Getting user account data

In [56]:
#returns the user data of all the reserves combined
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAccountData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'availableBorrowsUSD': 50.25262301932032,
 'borrowingPower': 0.0,
 'currentLiquidationThreshold': 0.65,
 'currentLtv': 0.5,
 'healthFactor': -1e-18,
 'healthFactorBelowThreshold': 0.0,
 'totalBorrowBalanceUSD': 0.0,
 'totalCollateralBalanceUSD': 100.50524603864064,
 'totalFeesUSD': 0.0,
 'totalLiquidityBalanceUSD': 100.50524603864064}


In [49]:
#returns the user data of all the reserves combined
params = {'_reserve': contracts['sample_token'],'_amount':10*10**18,"_fee":1,'_userCurrentBorrowBalanceUSD':0,'_userCurrentFeesUSD':0,'_userCurrentLtv':5*10**17}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("calculateCollateralNeededUSD")\
    .params(params)\
    .build()
response = icon_service.call(_call)
response = int(response,0)/10**18
# for key,values in response.items():
#     if key!="lastUpdateTimestamp":
#         response[key]=int(values,0)/10**18
#     else:
#         response[key]=int(values,0)
print(response)

20.0


### Getting user reserve data

In [57]:
#returns the data of a specific user for a specific reserve
#this returns the data of user for USDb reserve
params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' ,"_reserve":contracts['sicx']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    if key!="lastUpdateTimestamp":
        response[key]=int(values,0)/10**18
    else:
        response[key]=int(values,0)
pprint(response)

{'borrowRate': 0.0,
 'currentBorrowBalance': 0.0,
 'currentBorrowBalanceUSD': 0.0,
 'currentOTokenBalance': 201.01049207728127,
 'currentOTokenBalanceUSD': 100.56468866592635,
 'exchangeRate': 0.5002957190277554,
 'lastUpdateTimestamp': 0,
 'liquidityRate': 0.0,
 'originationFee': 0.0,
 'principalBorrowBalance': 0.0,
 'principalBorrowBalanceUSD': 0.0,
 'principalOTokenBalance': 201.01049207728127,
 'principalOTokenBalanceUSD': 100.56468866592635,
 'useAsCollateral': 1e-18,
 'userBorrowCumulativeIndex': 0.0}


# Getting All Reserve Data

In [214]:
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveData")\
    .build()
response = icon_service.call(_call)
pprint(response)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32032, 'message': 'REF_DATA_NOT_AVAILABLE'}


JSONRPCException: {'code': -32032, 'message': 'REF_DATA_NOT_AVAILABLE'} (Json rpc error)

### Getting the data of user for all reserves

In [42]:
#returns the data of a specific user for all reserves

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getUserAllReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        if k!="lastUpdateTimestamp":
            response[key][k]=int(v,0)/10**18
        else:
            response[key][k]=int(v,0)
pprint(response)

{'USDb': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0},
 'sICX': {'borrowRate': 0.0,
          'currentBorrowBalance': 0.0,
          'currentBorrowBalanceUSD': 0.0,
          'currentOTokenBalance': 0.0,
          'currentOTokenBalanceUSD': 0.0,
          'lastUpdateTimestamp': 0,
          'liquidityRate': 0.0,
          'originationFee': 0.0,
          'principalBorrowBalance': 0.0,
          'principalBorrowBalanceUSD': 0.0,
          'useAsCollateral': 0.0,
          'userBorrowCumulativeIndex': 0.0}}


### Get reserve configurations


In [61]:
#returns the configurations of a specific reserve
params = {"_reserve":contracts['sample_token']}
_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getReserveConfigurationData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    response[key]=int(values,0)
pprint(response)


{'baseLTVasCollateral': 330000000000000000,
 'borrowingEnabled': 1,
 'decimals': 18,
 'isActive': 1,
 'liquidationBonus': 10,
 'liquidationThreshold': 650000000000000000,
 'usageAsCollateralEnabled': 1}


### Get all reserves configurations

In [62]:
#returns the configurations of all the   reserves

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getAllReserveConfigurationData")\
    .build()
response = icon_service.call(_call)
for key,values in response.items():
    for k,v in values.items():
        response[key][k]=int(v,0)
pprint(response)


{'USDb': {'baseLTVasCollateral': 330000000000000000,
          'borrowingEnabled': 1,
          'decimals': 18,
          'isActive': 1,
          'liquidationBonus': 10,
          'liquidationThreshold': 650000000000000000,
          'usageAsCollateralEnabled': 1},
 'icx': {'baseLTVasCollateral': 330000000000000000,
         'borrowingEnabled': 1,
         'decimals': 18,
         'isActive': 1,
         'liquidationBonus': 10,
         'liquidationThreshold': 650000000000000000,
         'usageAsCollateralEnabled': 1}}


## Realtime USDb balance of user(with accrued interest)

In [33]:

params = {'_user': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['oToken'])\
    .method("balanceOf")\
    .params(params)\
    .build()
response = icon_service.call(_call)

print(int(response,0))

0


In [20]:

# params = {'_owner': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to(contracts['lendingPoolDataProvider'])\
    .method("getOracleAddress")\
    .build()
response = icon_service.call(_call)
print(response)

cx8397f621adac63d786c24cfdf920e683ae320dfd
